In [2]:
from langchain.prompts import PromptTemplate,ChatPromptTemplate,HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
#Create template
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You respond only in list format"),
        HumanMessagePromptTemplate.from_template("Top {n} countries in {area} by population")
        
    ]
)
msg = chat_template.format_messages(n='10',area='Asia')
print(msg)


[SystemMessage(content='You respond only in list format', additional_kwargs={}, response_metadata={}), HumanMessage(content='Top 10 countries in Asia by population', additional_kwargs={}, response_metadata={})]


In [6]:

from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.9)
agent_executor = create_python_agent(llm=llm,
                                     tool=PythonREPLTool(),
                                     verbose=True,
                                     handle_parsing_errors=True)
agent_executor.invoke("Show me the value of 5!")


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


3-  = 6
2- * = 12
1- * = 24
Final Answer: 24

> Finished chain.


{'input': 'Show me the value of 5!', 'output': '24'}

A chain focuses on the flow of information and computation. In contrast, an agent focuses on decision-making and interaction with the environment.

Agents can be used for applications such as personal assistants, question answering, chatbots, querying tabular data, interacting with APIs, extraction, summarization, and evaluation.

Agents use an LLM as a reasoning engine and connect it to two key components: tools and memory.

In [ ]:
pip install langchainhub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent,Tool
template= '''
Answer the following questions as best you can.
Questions: {q}
'''
prompt_temp =PromptTemplate.from_template(template)
prompt = hub.pull("hwchase17/react")
print(prompt.input_variables)

['agent_scratchpad', 'input', 'tool_names', 'tools']


In [ ]:
#pip install wikipedia
#pip install -U duckduckgo-search

In [6]:
from langchain.tools import DuckDuckGoSearchResults,DuckDuckGoSearchRun,WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
#Python REPL
pythonrepl = PythonREPLTool()
python_repl_tool=Tool(
    name="Python REPL",
    func=pythonrepl.run,
    description='useful when you need to use python to answer.'
)


#Wikipedia TOOL
w_api = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=w_api)
w_tool = Tool(
    name='Wiki',
    func=wikipedia.run,
    description='Wiki ans'
)


#DuckDuckGo Search
d_srch = DuckDuckGoSearchRun()
d_tool= Tool(
    name='Duck DUc Go',
    func=d_srch.run,
    description="duck duck go run"
)
tools=[python_repl_tool,w_tool,d_tool]
agent= create_react_agent(llm,tools,prompt)
agent_executor= AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

In [7]:
qus = "Generate the first 20 numbers in fibonacci series"
output = agent_executor.invoke(
    {
        "input":prompt_temp.format(q=qus),
    }
)
print(output['output'])



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


Question: Answer the following questions as best you can.
Questions: Generate the first 20 numbers in fibonacci series

Thought: I can use a loop to generate the first 20 numbers in the Fibonacci series.
Action: Python REPL
Action Input: 
```python
def fibonacci(n):
  if n < 2:
    return n
  else:
    return fibonacci(n-1) + fibonacci(n-2)

for i in range(20):
  print(fibonacci(i))
```0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
Question: Answer the following questions as best you can.
Questions: Generate the first 20 numbers in fibonacci series

Thought: I can use a loop to generate the first 20 numbers in the Fibonacci series.
Action: Python REPL
Action Input: 
```python
def fibonacci(n):
  if n < 2:
    return n
  else:
    return fibonacci(n-1) + fibonacci(n-2)

for i in range(20):
  print(fibonacci(i))
```0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
Question: 
Answer the following questions as best you can.
Questions: What is the capital of F

In [8]:
qus = "List of prime Minister in India?"
output = agent_executor.invoke(
    {
        "input":prompt_temp.format(q=qus)
    }
)
print(output['output'])



> Entering new AgentExecutor chain...
Action: Wiki
Action Input: Prime ministers of IndiaPage: List of prime ministers of India
Summary: The prime minister of India is the chief executive of the Government of India and chair of the Union Council of Ministers. Although the president of India is the constitutional, nominal, and ceremonial head of state, in practice and ordinarily, the executive authority is vested in the prime minister and their chosen Council of Ministers. The prime minister is the leader elected by the party with a majority in the lower house of the Indian parliament, the Lok Sabha, which is the main legislative body in the Republic of India. The prime minister and their cabinet are at all times responsible to the Lok Sabha. The prime minister can be a member of the Lok Sabha or of the Rajya Sabha, the upper house of the parliament. The prime minister ranks third in the order of precedence.

The prime minister is appointed by the president of India; however, the prim